In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt

**1 Noise Power Estimation**
In this part of the exercise, the noise power is estimated based on the speech presence probability. First, use
your functions from the second exercise to create the STFT of the noisy input signals. The frame length should
be 32 ms and the frame shift 16 ms. Use a √Hann-window as analysis window. From the STFT, compute the
periodograms and determine the noise PSD by performing the following steps for each frame.

Look at sheet for three steps. (Complicated formulas)

**Questions**

1.1 In equation (1), (3) and (4) the noise power estimate from the previous frame, ˆσ2n[k,` −1], is required.
However, there is none available for the first frame ` = 0. The same problem occurs for the smoothed speech
presence probability Q[k,`] in (2).

- What would be appropriate initializations for the first noise estimate ˆσ2n[k,−1] and the smoothed
posterior probability Q[k,−1]? Explain briefly why you chose your initialization method.

1.2 Plot the speech presence probability P(H1|Y [k,`]) using plt.imshow.

a) Which values do you obtain for time-frequency points where speech is present?

b) What values do you get for time-frequency points where only noise is present?

c) If you compare the speech presence probability with the spectrogram of your input signal, can you see
similarities?

1.3 Plot the estimated noise PSD as a spectrogram using plt.imshow.

a) How well is the background noise estimated?

b) Can you observe errors (e. g. components that do not belong to the background noise)?

c) What would be the consequence of such errors?

**2 A priori SNR estimation and Wiener Filtering**

Perform the steps in equations (5) – (7) for every frame of your input signal and store all enhanced speech
spectra ˆS[k,`] in a matrix, e. g. m_enhanced_stft. (Look at sheet for the steps)

*Questions*

2.1 In equation (5), the issue from Section 1 occurs again. An initialization for ˆS[k,`] is required for the first
frame.

*What would be a reasonable choice in this case?*

2.2 Set α = 0.5 and Gmin = 0. Plot the magnitude spectrogram of the noisy speech signal and the enhanced
speech signal in dB and compare both. Make sure, that the color bar for both plots is the same. This can
be achieved by manually setting the vmin and vmax parameters of plt.imshow.

a) How does the clean spectrogram differ from the spectrogram of the noisy input signal?

b) Can you see artifacts in the spectrogram of the enhanced signal?

**3 Parameter tuning**

Use the compute_istft function to synthesize the enhanced speech signal. For this, employ the same frame shift
and FFT length as in Section 1. Further, use the √Hann-window also for the synthesis.

Listen to the noisy signal and the enhanced signal. For this you could use the play function from sounddevice.

*Questions*

3.1 Compare the noisy and the enhanced signal with each other.

a) How well is the background noise suppressed?

b) Can you hear any distortions of the speech signal in the enhanced signal?

c) What artifacts can you hear?

3.2 Vary α between 0 and 1 and listen to the synthesized signals.

a) What differences can you perceive?

b) How do the artifacts, speech signal and noise suppression change?

c) What is your favorite setting? Explain why.

d) Are the differences you heard also visible in the spectrogram of the enhanced speech signal?

3.3 Try different values for Gmin which can be varied between 0 and 1, i. e. between −∞ dB and 0 dB. Listen
again to the synthesized signals.

a) What differences can be perceived now?

b) How does this parameter affect the artifacts, speech signal and noise suppression and what would be
your favorite setting this time? Again, explain why.

c) How do the spectrograms change in this case?

In your report, include some spectrograms that support your reasoning for your choice of the parameters α and
Gmin.
